(C:filtres-numeriques)=
# Filtres numériques

Les filtres numériques sont très courants et utilisés dans de nombreux domaines :
télécommunications, traitement de la parole, Hi-Fi, radar et sonar, asservissement, vidéo, photos, modem, etc.

S'ils sont à ce point répandus, c'est en raison de leurs nombreux avantages par rapport aux filtres analogiques :
* deux filtres numériques de même configuration sont parfaitement identiques,
* un filtre numérique est facilement reprogrammable,
* l'association de filtres numériques ne pose pas de problème d'impédance,
* il n'y a pas de variation des caractéristiques en fonction de l'âge ou de la température des composants,
* on dispose d'une grande précision de calcul.

En revanche, les filtres numériques sont bien entendu limités par leur fréquence d'échantillonnage
et naturellement inadaptés au traitement du signal analogique.

La réponse impulsionnelle d'un filtre numérique est échantillonnée à une fréquence d'échantillonnage $f_e$.

Dans ce cours, on traitera uniquement des filtres causaux :

$$
h[n] = 0\;\text{si}\; n<0.
$$

Par ailleurs, lorsque la réponse impulsionnelle est à durée limitée, c'est à dire $h[n]=0$ si $n > N$,
alors le filtre est à {ref}`réponse impulsionnelle finie (RIF)<C:filtres-rif>`.
Dans le cas contraire, on parle de filtre à {ref}`réponse impulsionnelle infinie (RII)<C:filtres-rii>`.
Ces deux types de filtres sont présentés ci-après, ainsi que deux méthodes de conception :
* la méthode des fenêtres pour les filtres RIF,
* la transformation bilinéaire pour les filtres RII.